# Setup

In [1]:
!pip install pyspark

In [2]:
from pyspark.ml.classification import LinearSVC, GBTClassifier, LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.ml.linalg import VectorUDT, Vectors
from pyspark.sql import types as t
import numpy as np

In [3]:
spark = (
    SparkSession.builder.config("spark.executor.memory", "4g")
    .config("spark.executor.cores", "2")
    .config("spark.cores.max", "2")
    .config("spark.driver.memory", "8g")
    .getOrCreate()
)

# Read the dataset(s)

Make sure to upload the train datasets generated by the `make-dataset.sh` script before proceeding.
Either train using X4 or X3,X2

In [26]:
!unzip X4.zip

Archive:  X4.zip
replace X4.parquet/.part-00000-34b7377e-848c-4866-bfc8-33e858f2cd38-c000.snappy.parquet.crc? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [4]:
df = spark.read.parquet('./X4.parquet')

In [6]:
df1 = spark.read.parquet('./X2.parquet')
df2 = spark.read.parquet('./X3.parquet')
df = df1.unionAll(df2)
df.toPandas()

,features,label
0,"[0.32666666666666666, 1.0, 1.0, 1.0, 0.2407407...",1
1,"[0.5026178010471204, 1.0, 1.0, 0.0, 0.0, 0.0, ...",1
2,"[0.17842323651452285, 1.0, 1.0, 1.0, 0.3611111...",1
3,"[0.7969348659003832, 1.0, 1.0, 1.0, 1.0, 0.0, ...",1
4,"[0.4024896265560166, 1.0, 1.0, 1.0, 0.24074074...",1
...,...,...
230533,"[0.2574850299401198, 1.0, 1.0, 0.0, 0.21428571...",0
230534,"[0.2857142857142857, 1.0, 1.0, 0.0, 0.28205128...",0
230535,"[0.2544378698224852, 1.0, 0.0, 0.0, 0.07999999...",0
230536,"[0.2544378698224852, 1.0, 1.0, 0.0, 0.21428571...",0


In [5]:
def stratified_split_train_test(df, frac, label,seed=42):
    fractions = df.select(label).distinct().withColumn("fraction", f.lit(frac)).rdd.collectAsMap()
    df_frac = df.stat.sampleBy(label, fractions, seed)
    df_remaining = df.exceptAll(df_frac)
    return df_frac, df_remaining

train_set, test_set = stratified_split_train_test(df, 0.8, 'label')

def with_weights(df, column="label"):
    w_zero = 1 / df.filter(f.col(column) == 0).count()
    w_one = 0.5 / df.filter(f.col(column) == 1).count()
    return df.withColumn("weight", f.when(f.col(column) == 0, w_zero).otherwise(w_one))

train_set = with_weights(train_set)
train_set_base, train_set_comb = stratified_split_train_test(train_set, 0.6, 'label', seed=5476)

# Train the model with ensemble stacking

In [6]:
model_svc = LinearSVC(weightCol='weight', maxIter=100, predictionCol='prediction_svc')
paramGrid = ParamGridBuilder()\
    .addGrid(model_svc.threshold, [1.3,1.4,1.5])\
    .build()
cvs = CrossValidator(estimator=model_svc,
                     estimatorParamMaps=paramGrid,
                     evaluator=BinaryClassificationEvaluator(weightCol='weight',metricName='areaUnderPR'),
                     numFolds=4)
estimator_svc = cvs.fit(train_set_base)

In [7]:
model_gbt = GBTClassifier(weightCol='weight', maxBins=8, maxIter=100, predictionCol='prediction_gbt', lossType='logistic')
paramGrid = ParamGridBuilder()\
    .addGrid(model_gbt.maxDepth, [4,8])\
    .build()

cvs = TrainValidationSplit(estimator=model_gbt,
                           estimatorParamMaps=paramGrid,
                     evaluator=BinaryClassificationEvaluator(weightCol='weight', metricName='areaUnderPR'),
                     seed=1234,
                     trainRatio=0.8
                    )

estimator_gbt = cvs.fit(train_set_base)

In [8]:
interm = estimator_svc.transform(train_set_comb).drop('rawPrediction').drop('probability')
interm = estimator_gbt.transform(interm).drop('rawPrediction').drop('probability')
interm.groupby('prediction_svc', 'prediction_gbt').count().show()

+--------------+--------------+------+
|prediction_svc|prediction_gbt| count|
+--------------+--------------+------+
|           1.0|           1.0|   249|
|           0.0|           1.0|  5822|
|           1.0|           0.0|    15|
|           0.0|           0.0|216771|
+--------------+--------------+------+



In [9]:
interm = interm.drop('features').select('label', 'weight','prediction_svc', 'prediction_gbt')
interm.show()

+-----+--------------------+--------------+--------------+
|label|              weight|prediction_svc|prediction_gbt|
+-----+--------------------+--------------+--------------+
|    0|1.816731735941675...|           0.0|           0.0|
|    0|1.816731735941675...|           0.0|           0.0|
|    0|1.816731735941675...|           0.0|           0.0|
|    0|1.816731735941675...|           0.0|           0.0|
|    0|1.816731735941675...|           0.0|           0.0|
|    0|1.816731735941675...|           0.0|           0.0|
|    0|1.816731735941675...|           0.0|           0.0|
|    0|1.816731735941675...|           0.0|           0.0|
|    0|1.816731735941675...|           0.0|           0.0|
|    0|1.816731735941675...|           0.0|           0.0|
|    0|1.816731735941675...|           0.0|           0.0|
|    0|1.816731735941675...|           0.0|           0.0|
|    0|1.816731735941675...|           0.0|           0.0|
|    0|1.816731735941675...|           0.0|           0.

In [11]:
interm = interm.withColumn('prediction_svc', f.col('prediction_svc').cast(t.IntegerType()))
interm = interm.withColumn('prediction_gbt', f.col('prediction_gbt').cast(t.IntegerType()))

@f.udf(returnType=VectorUDT())
def toVec(p1,p2):
    l = [p1,p2]
    return Vectors.dense(l)

interm = interm.withColumn('features', toVec(f.col('prediction_svc'), f.col('prediction_gbt')))
interm.show()

+-----+--------------------+--------------+--------------+---------+
|label|              weight|prediction_svc|prediction_gbt| features|
+-----+--------------------+--------------+--------------+---------+
|    0|1.816731735941675...|             0|             0|[0.0,0.0]|
|    0|1.816731735941675...|             0|             0|[0.0,0.0]|
|    0|1.816731735941675...|             0|             0|[0.0,0.0]|
|    0|1.816731735941675...|             0|             0|[0.0,0.0]|
|    0|1.816731735941675...|             0|             0|[0.0,0.0]|
|    0|1.816731735941675...|             0|             0|[0.0,0.0]|
|    0|1.816731735941675...|             0|             0|[0.0,0.0]|
|    0|1.816731735941675...|             0|             0|[0.0,0.0]|
|    0|1.816731735941675...|             0|             0|[0.0,0.0]|
|    0|1.816731735941675...|             0|             0|[0.0,0.0]|
|    0|1.816731735941675...|             0|             0|[0.0,0.0]|
|    0|1.816731735941675...|      

In [ ]:
model_lr = LogisticRegression(predictionCol='prediction', featuresCol='features',weightCol='weight', maxIter=100)
paramGrid = ParamGridBuilder()\
    .addGrid(model_lr.regParam, [0.5,0.25])\
    .addGrid(model_lr.threshold, [0.51])\
    .build()
cvs = CrossValidator(estimator=model_lr,
                     estimatorParamMaps=paramGrid,
                     evaluator=BinaryClassificationEvaluator(metricName='areaUnderPR'),
                     numFolds=4)
estimator_lr = cvs.fit(interm)

# Prediction and evaluation

In [ ]:
prediction = estimator_svc.transform(test_set).drop('rawPrediction').drop('probability')
prediction = estimator_gbt.transform(prediction).drop('rawPrediction').drop('probability')
prediction = prediction.drop('features')
prediction = prediction.withColumn('features', toVec(f.col('prediction_svc'), f.col('prediction_gbt')))
prediction = estimator_lr.transform(prediction)
prediction.toPandas()

In [ ]:
accuracy = prediction.filter(f.col('label') == f.col('prediction')).count() / prediction.count()
print("Accuracy: ", accuracy)

tp = prediction.filter("label==1 AND prediction==1").count() 
p = tp / prediction.filter('prediction==1').count()
r = tp / prediction.filter('label == 1').count()
f1 = 2 * p * r / (p + r)
print("F1 score: ", f1)

In [21]:
estimator_svc.save("model-notebooks_1")
estimator_gbt.save("model-notebooks_2")
estimator_lr.save("model-notebooks_comb")

In [ ]:
estimator_svc.save("model-products_1")
estimator_gbt.save("model-products_2")
estimator_lr.save("model-products_comb")

In [24]:
!zip -r models.zip model-notebooks_1 model-notebooks_2 model-notebooks_comb# model-products

	zip warning: name not matched: model-notebooks_comb#
updating: model-products/ (stored 0%)
updating: model-products/data/ (stored 0%)
updating: model-products/data/._SUCCESS.crc (stored 0%)
updating: model-products/data/_SUCCESS (stored 0%)
updating: model-products/metadata/ (stored 0%)
updating: model-products/metadata/part-00000 (deflated 46%)
updating: model-products/metadata/._SUCCESS.crc (stored 0%)
updating: model-products/metadata/.part-00000.crc (stored 0%)
updating: model-products/metadata/_SUCCESS (stored 0%)
updating: model-products/data/part-00000-197a58fc-8370-407c-9ee5-66efea0bce84-c000.snappy.parquet (deflated 43%)
updating: model-products/data/.part-00000-197a58fc-8370-407c-9ee5-66efea0bce84-c000.snappy.parquet.crc (stored 0%)
updating: model-notebooks_1/ (stored 0%)
updating: model-notebooks_1/bestModel/ (stored 0%)
updating: model-notebooks_1/bestModel/data/ (stored 0%)
updating: model-notebooks_1/bestModel/data/._SUCCESS.crc (stored 0%)
updating: model-notebooks_1/b

In [ ]:
!zip -r models.zip model-products_1 model-products_2 model-products_comb